In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
import os.path

class main(object):
    
    def __init__(self, n_class, len_seq, dim_mfcc, size_batch, learning_rate, step_train, path_tfr_train,):
        self.n_class = n_class
        self.len_seq = len_seq
        self.dim_mfcc = dim_mfcc
        self.size_batch = size_batch
        self.learning_rate = learning_rate
        self.step_train = step_train
        self.path_tfr_train = path_tfr_train

    #TFRecordsからデータセット取り出し
    def input(self, path_tfr):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)

        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })
        
        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)
        
        datas = tf.reshape(datas, [self.len_seq, self.dim_mfcc])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.shuffle_batch(
            [datas, labels],
            batch_size=self.size_batch, capacity=1000+self.size_batch*self.dim_mfcc,
            min_after_dequeue=1000
        )
            
        return datas, labels

    #RNNモデル
    def RNN(self, x):
        x = tf.unstack(x, self.len_seq, 1)
        lstm_cell = rnn.BasicLSTMCell(256, forget_bias=1.0)
        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

        weight = tf.Variable(tf.random_normal([256, self.n_class]))
        bias = tf.Variable(tf.random_normal([self.n_class]))

        return tf.matmul(outputs[-1], weight) + bias
    
    #トレーニング
    def train(self):
        x = tf.placeholder(tf.float32, shape=[None, self.len_seq, self.dim_mfcc])
        y_ = tf.placeholder(tf.float32, shape=[None, self.n_class])

        preds = self.RNN(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y_))
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(cost)
        correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        datas, labels = self.input(self.path_tfr_train)
        labels = tf.one_hot(labels, depth=self.n_class, dtype=tf.float32)
        
        n_training_iters = self.step_train * self.size_batch
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        
        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            step = 1
            while step * self.size_batch < n_training_iters:
                batch = sess.run([datas, labels])
                batch[1] = batch[1].reshape([-1, self.n_class])
                sess.run(optimizer, feed_dict={x: batch[0], y_:batch[1]})
                if step % 100 == 0:
                    acc = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
                    loss = sess.run(cost, feed_dict={x: batch[0], y_: batch[1]})
                    print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
                step += 1

            """
            # テスト
            test_len = 128
            test_batch = self.input(os.path.join(self.PATH_DATASET, 'test-male.tfrecords'), self.MFCC_DIM, test_len)
            test_data, test_label = sess.run(test_batch)
            test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
            print("Test Accuracy: {}".format(test_acc))
            """
            coord.request_stop()
            coord.join(threads)

In [2]:
m = main(
    n_class = 22,
    len_seq = 300,
    dim_mfcc = 39,
    size_batch = 30,
    learning_rate = 0.001,
    step_train = 10000,
    path_tfr_train = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset/train-male.tfrecords'
    )
m.train()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step: 100 / loss: 3.070546 / acc: 0.06667
step: 200 / loss: 3.116491 / acc: 0.03333
step: 300 / loss: 3.114882 / acc: 0.03333
step: 400 / loss: 3.104908 / acc: 0.06667
step: 500 / loss: 3.137856 / acc: 0.03333
step: 600 / loss: 3.017043 / acc: 0.13333
step: 700 / loss: 3.086022 / acc: 0.06667
step: 800 / loss: 3.089572 / acc: 0.03333
step: 900 / loss: 3.099506 / acc: 0.00000
step: 1000 / loss: 3.094058 / acc: 0.03333
step: 1100 / loss: 3.082890 / acc: 0.10000
step: 1200 / loss: 3.080096 / acc: 0.03333
step: 1300 / loss: 3.063076 / acc: 0.03333
step: 1400 / loss: 3.102589 / acc: 0.03333
step: 1500 / loss: 3.036189 / acc: 0.00000
step: 1600 / loss: 3.050397 / acc: 0.13333
step: 1700 / loss: 3.136124 / acc: 0.03333
step: 1800 / loss: 3.115227 / acc: 0.03333
step: 1900 / loss: 3.093938 / acc: 0.06667
